In [1]:
# train_swin_tiny_v3.py
import os
import time
import random
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import glob
import cv2

# ------------------------
# ⚙️ Device & Seed
# ------------------------
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔥 Using {device} ({torch.cuda.get_device_name(0) if device.type=='cuda' else 'CPU'})")

# ------------------------
# 📁 Paths
# ------------------------
BASE = "/data/ephemeral/home/data"
DATA_DIR = f"{BASE}/processed/stage0_5_train/"
META_PATH = f"{BASE}/meta_stage0_5_train.csv"
TRAIN_CSV = f"{BASE}/raw/train.csv"
TEST_PATH = f"{BASE}/processed/stage0_5_test/"
SUB_PATH = f"{BASE}/raw/sample_submission.csv"

# ------------------------
# 🧩 Dataset
# ------------------------
class ImageDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = np.array(Image.open(row["filepath"]).convert("RGB"))
        target = int(row["target"])
        if self.transform:
            img = self.transform(image=img)["image"]
        return img, target

# ------------------------
# ⚙️ Config
# ------------------------
model_name = "swin_tiny_patch4_window7_224"
IMG_SIZE = 224
NUM_CLASSES = 17
LR = 3e-4
EPOCHS = 30
BATCH_SIZE = 64
num_workers = 8

# ------------------------
# 🧩 Albumentations Transforms
# ------------------------
trn_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.OneOf([
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.2),
    ], p=0.6),
    A.Rotate(limit=25, border_mode=cv2.BORDER_REFLECT_101, p=0.4),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.08, rotate_limit=10, p=0.4),
    A.Perspective(scale=(0.05, 0.10), p=0.2),
    A.OneOf([
        A.MotionBlur(blur_limit=5, p=0.3),
        A.GaussianBlur(blur_limit=3, p=0.3),
        A.GaussNoise(var_limit=(5, 25), p=0.3),
    ], p=0.3),
    A.OneOf([
        A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.4),
        A.RandomShadow(p=0.2),
    ], p=0.3),
    A.HueSaturationValue(hue_shift_limit=8, sat_shift_limit=10, val_shift_limit=10, p=0.25),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

tst_transform = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

# ------------------------
# 📈 Data Split
# ------------------------
meta = pd.read_csv(META_PATH)
train_csv = pd.read_csv(TRAIN_CSV)

meta["basename"] = meta["filepath"].apply(lambda x: os.path.basename(x).split("_", 1)[-1])
train_csv["basename"] = train_csv["ID"].apply(lambda x: os.path.basename(x))
meta_joined = pd.merge(meta, train_csv[["basename", "target"]], on="basename", how="left")
meta_joined = meta_joined.dropna(subset=["target"]).reset_index(drop=True)

trn_df, val_df = train_test_split(
    meta_joined, test_size=0.2, stratify=meta_joined["target"], random_state=SEED
)
print("✅ Split 완료:", trn_df.shape, val_df.shape)

# ------------------------
# 🧱 Dataloaders
# ------------------------
trn_loader = DataLoader(ImageDataset(trn_df, trn_transform),
                        batch_size=BATCH_SIZE, shuffle=True,
                        num_workers=num_workers, pin_memory=True)
val_loader = DataLoader(ImageDataset(val_df, tst_transform),
                        batch_size=BATCH_SIZE, shuffle=False,
                        num_workers=num_workers, pin_memory=True)

# ------------------------
# 🧠 Model
# ------------------------
model = timm.create_model(model_name, pretrained=True, num_classes=NUM_CLASSES,
                          in_chans=3, drop_path_rate=0.1).to(device)

loss_fn = nn.CrossEntropyLoss(label_smoothing=0.05)
optimizer = AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS, eta_min=1e-5)
scaler = torch.cuda.amp.GradScaler()

# ------------------------
# 🧮 Train / Validate
# ------------------------
def train_one_epoch(loader, model, optimizer, loss_fn, device, scaler=None):
    model.train()
    total_loss, preds_list, targets_list = 0, [], []
    for images, targets in tqdm(loader, desc="Train", leave=False):
        images, targets = images.to(device), targets.to(device)
        optimizer.zero_grad(set_to_none=True)
        with torch.amp.autocast("cuda", enabled=(device.type == "cuda")):
            preds = model(images)
            loss = loss_fn(preds, targets)
        if scaler:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()
        total_loss += loss.item()
        preds_list.extend(preds.argmax(1).detach().cpu().numpy())
        targets_list.extend(targets.cpu().numpy())
    return {
        "loss": total_loss / len(loader),
        "acc": accuracy_score(targets_list, preds_list),
        "f1": f1_score(targets_list, preds_list, average="macro"),
    }

@torch.no_grad()
def validate(loader, model, loss_fn, device):
    model.eval()
    total_loss, preds_list, targets_list = 0, [], []
    for images, targets in tqdm(loader, desc="Valid", leave=False):
        images, targets = images.to(device), targets.to(device)
        with torch.amp.autocast("cuda"):
            preds = model(images)
            loss = loss_fn(preds, targets)
        total_loss += loss.item()
        preds_list.extend(preds.argmax(1).cpu().numpy())
        targets_list.extend(targets.cpu().numpy())
    return {
        "loss": total_loss / len(loader),
        "acc": accuracy_score(targets_list, preds_list),
        "f1": f1_score(targets_list, preds_list, average="macro"),
    }

# ------------------------
# 🚀 Training Loop
# ------------------------
best_f1 = 0.0
for epoch in range(EPOCHS):
    train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, scaler)
    val_metrics = validate(val_loader, model, loss_fn, device)
    scheduler.step()

    print(f"\n[Epoch {epoch+1}/{EPOCHS}] "
          f"Train | Loss {train_metrics['loss']:.4f} Acc {train_metrics['acc']:.4f} F1 {train_metrics['f1']:.4f} | "
          f"Valid | Loss {val_metrics['loss']:.4f} Acc {val_metrics['acc']:.4f} F1 {val_metrics['f1']:.4f}")

    if val_metrics["f1"] > best_f1:
        best_f1 = val_metrics["f1"]
        torch.save(model.state_dict(), f"./best_swin_tiny_v3_{best_f1:.4f}.pt")
        print(f"✅ Best model saved (F1={best_f1:.4f})")

# ------------------------
# 🧾 Inference (TTA)
# ------------------------
print("\n🚀 Inference with rotation TTA...")
model.load_state_dict(torch.load(f"./best_swin_tiny_v3_{best_f1:.4f}.pt"))
model.eval()

sub = pd.read_csv(SUB_PATH)
preds_list = []

angles = [0, 90, 180, 270]

for name in tqdm(sub["ID"], desc="Inference"):
    matches = glob.glob(os.path.join(TEST_PATH, "**", name), recursive=True)
    if not matches:
        raise FileNotFoundError(f"테스트 이미지 없음: {name}")
    img = np.array(Image.open(matches[0]).convert("RGB"))

    tta_preds = []
    for ang in angles:
        rotated = cv2.rotate(img, {
            0: cv2.ROTATE_90_CLOCKWISE,
            90: cv2.ROTATE_180,
            180: cv2.ROTATE_90_COUNTERCLOCKWISE,
            270: None
        }[ang]) if ang in [0, 90, 180] else img

        tensor = tst_transform(image=rotated)["image"].unsqueeze(0).to(device)
        with torch.no_grad():
            pred = torch.softmax(model(tensor), dim=1)
        tta_preds.append(pred.cpu().numpy())

    mean_pred = np.mean(tta_preds, axis=0)
    preds_list.append(np.argmax(mean_pred))

sub["target"] = preds_list
sub.to_csv("v3_swin_tiny_split.csv", index=False)
print("🎯 Inference complete! Saved to v3_swin_tiny_split.csv")


🔥 Using cuda (NVIDIA GeForce RTX 3090)
✅ Split 완료: (1256, 4) (314, 4)


/data/ephemeral/home/py310/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_338998/3922242674.py:89: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5, 25), p=0.3),
/tmp/ipykernel_338998/3922242674.py:143: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
                                                                                                                                                


[Epoch 1/30] Train | Loss 1.8524 Acc 0.4443 F1 0.4057 | Valid | Loss 1.0766 Acc 0.6879 F1 0.6227
✅ Best model saved (F1=0.6227)



[Epoch 2/30] Train | Loss 1.0728 Acc 0.7110 F1 0.6830 | Valid | Loss 0.7481 Acc 0.8248 F1 0.7832
✅ Best model saved (F1=0.7832)



[Epoch 3/30] Train | Loss 0.8732 Acc 0.7914 F1 0.7625 | Valid | Loss 0.7052 Acc 0.8376 F1 0.7981
✅ Best model saved (F1=0.7981)


...


[Epoch 28/30] Train | Loss 0.3885 Acc 0.9777 F1 0.9743 | Valid | Loss 0.5111 Acc 0.9204 F1 0.9171



[Epoch 29/30] Train | Loss 0.3889 Acc 0.9777 F1 0.9768 | Valid | Loss 0.5128 Acc 0.9268 F1 0.9237



[Epoch 30/30] Train | Loss 0.4085 Acc 0.9729 F1 0.9708 | Valid | Loss 0.5132 Acc 0.9204 F1 0.9178

🚀 Inference with rotation TTA...


Inference: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 3140/3140 [02:06<00:00, 24.78it/s]

🎯 Inference complete! Saved to v3_swin_tiny_split.csv
